In [29]:
import basis_set_exchange as bse
import json
# Obtain the STO-3G basis set in nwchem format (as a string) for hydrogen and carbon
basis = json.loads(bse.get_basis('3-21G', elements=[10], fmt='json', header=False, uncontract_general=True))

In [31]:
neon_basis = basis["elements"]["10"]

In [34]:
for a in neon_basis["electron_shells"]:
    print(a)

{'function_type': 'gto', 'region': 'valence', 'angular_momentum': [0], 'exponents': ['0.5157240000E+03', '0.7765380000E+02', '0.1681360000E+02'], 'coefficients': [['0.5814303044E-01', '0.3479511822E+00', '0.7107143721E+00']]}
{'function_type': 'gto', 'region': 'valence', 'angular_momentum': [0, 1], 'exponents': ['0.1248300000E+02', '0.2664510000E+01'], 'coefficients': [['-0.4099223208E+00', '0.1224310958E+01'], ['0.2474599836E+00', '0.8517429435E+00']]}
{'function_type': 'gto', 'region': 'valence', 'angular_momentum': [0, 1], 'exponents': ['0.6062500000E+00'], 'coefficients': [['0.1000000000E+01'], ['0.1000000000E+01']]}


In [60]:
neon_basis["electron_shells"][1]

{'function_type': 'gto',
 'region': 'valence',
 'angular_momentum': [0, 1],
 'exponents': ['0.1248300000E+02', '0.2664510000E+01'],
 'coefficients': [['-0.4099223208E+00', '0.1224310958E+01'],
  ['0.2474599836E+00', '0.8517429435E+00']]}

In [48]:
from vampyr import vampyr3d as vp

g_exp = vp.GaussExp() # Create a GaussExp object

angular_momentum = neon_basis["electron_shells"][0]["angular_momentum"]
angular_momentum = [int(a) for a in angular_momentum]
exponents = neon_basis["electron_shells"][0]["exponents"]
exponents = [float(a) for a in exponents]
coefficients = neon_basis["electron_shells"][0]["coefficients"]
coefficients = [[float(a) for a in orb_type] for orb_type in coefficients]

s_poly = [0, 0, 0]
p_poly = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
d_poly = [[2, 0, 0], [0, 2, 0], [0, 0, 2], [1, 1, 0], [1, 0, 1], [0, 1, 1]]


In [53]:
inputs = []
for i in range(len(exponents)):
    inputs.append([exponents[i], coefficients[0][i], [0, 0, 0], s_poly])

g_exp = vp.GaussExp()
for inp in inputs:
    g_exp.append(vp.GaussFunc(*inp))

In [120]:
# Function that makes list of GuassFunc objects
# from basis set exchange data s-orbitals

def make_gaussfuncs_s(exponents, coefficients, pos=[0, 0, 0]):
    s_poly = [0, 0, 0]
    s_gaussFuncs = []
    for i in range(len(exponents)):
        s_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[0][i], pos, s_poly))
    return s_gaussFuncs

def make_gaussfuncs_p(exponents, coefficients, pos=[0, 0, 0]):
    p_poly = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    p_gaussFuncs = []
    for i in range(len(exponents)):
        p_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[1][i], pos, p_poly[0]))
        p_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[1][i], pos, p_poly[1]))
        p_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[1][i], pos, p_poly[2]))
    return p_gaussFuncs

def make_gaussfuncs_d(exponents, coefficients, pos=[0, 0, 0]):
    d_poly = [[2, 0, 0], [0, 2, 0], [0, 0, 2], [1, 1, 0], [1, 0, 1], [0, 1, 1]]
    d_gaussFuncs = []
    for i in range(len(exponents)):
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[0]))
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[1]))
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[2]))
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[3]))
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[4]))
        d_gaussFuncs.append(vp.GaussFunc(exponents[i], coefficients[2][i], pos, d_poly[5]))
    return d_gaussFuncs


In [110]:
neon_basis["electron_shells"]

[{'function_type': 'gto',
  'region': 'valence',
  'angular_momentum': [0],
  'exponents': ['0.5157240000E+03', '0.7765380000E+02', '0.1681360000E+02'],
  'coefficients': [['0.5814303044E-01',
    '0.3479511822E+00',
    '0.7107143721E+00']]},
 {'function_type': 'gto',
  'region': 'valence',
  'angular_momentum': [0, 1],
  'exponents': ['0.1248300000E+02', '0.2664510000E+01'],
  'coefficients': [['-0.4099223208E+00', '0.1224310958E+01'],
   ['0.2474599836E+00', '0.8517429435E+00']]},
 {'function_type': 'gto',
  'region': 'valence',
  'angular_momentum': [0, 1],
  'exponents': ['0.6062500000E+00'],
  'coefficients': [['0.1000000000E+01'], ['0.1000000000E+01']]}]

In [114]:
def basis_to_list_of_gaussFuncs(basis):
    from vampyr import vampyr3d as vp
    list_of_gaussFuncs = []
    for i in range(len(basis["electron_shells"])):
        angular_momentum = neon_basis["electron_shells"][i]["angular_momentum"]
        angular_momentum = [int(a) for a in angular_momentum]
        exponents = neon_basis["electron_shells"][i]["exponents"]
        exponents = [float(a) for a in exponents]
        coefficients = neon_basis["electron_shells"][i]["coefficients"]
        coefficients = [[float(a) for a in orb_type] for orb_type in coefficients]
        for mom in angular_momentum:
            if mom == 0:
                list_of_gaussFuncs.append(make_gaussfuncs_s(exponents, coefficients, pos=[0, 0, 0]))
            if mom == 1:
                list_of_gaussFuncs.append(make_gaussfuncs_p(exponents, coefficients, pos=[0, 0, 0]))
            if mom == 2:
                list_of_gaussFuncs.append(make_gaussfuncs_d(exponents, coefficients, pos=[0, 0, 0]))
        
    return list_of_gaussFuncs

In [115]:
basis_list = basis_to_list_of_gaussFuncs(neon_basis)

0
1
2


In [119]:
neon_basis["electron_shells"][1]

{'function_type': 'gto',
 'region': 'valence',
 'angular_momentum': [0, 1],
 'exponents': ['0.1248300000E+02', '0.2664510000E+01'],
 'coefficients': [['-0.4099223208E+00', '0.1224310958E+01'],
  ['0.2474599836E+00', '0.8517429435E+00']]}

In [116]:
for l in basis_list:
    for g in l:
        print(g)

Coef    : 0.058143
Exp     : 515.724
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : 0.347951
Exp     : 77.6538
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : 0.710714
Exp     : 16.8136
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : -0.409922
Exp     : 12.483
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : 1.22431
Exp     : 2.66451
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : 0.24746
Exp     : 12.483
Pos     : 0 0 0 
Pow     : 1 0 0 

Coef    : 0.24746
Exp     : 12.483
Pos     : 0 0 0 
Pow     : 0 1 0 

Coef    : 0.24746
Exp     : 12.483
Pos     : 0 0 0 
Pow     : 0 0 1 

Coef    : 0.851743
Exp     : 2.66451
Pos     : 0 0 0 
Pow     : 1 0 0 

Coef    : 0.851743
Exp     : 2.66451
Pos     : 0 0 0 
Pow     : 0 1 0 

Coef    : 0.851743
Exp     : 2.66451
Pos     : 0 0 0 
Pow     : 0 0 1 

Coef    : 1
Exp     : 0.60625
Pos     : 0 0 0 
Pow     : 0 0 0 

Coef    : 1
Exp     : 0.60625
Pos     : 0 0 0 
Pow     : 1 0 0 

Coef    : 1
Exp     : 0.60625
Pos     : 0 0 0 
Pow     : 0 1 0 

Coef    : 1
Exp     